In [91]:
import numpy as np
import pandas as pd
from sklearn import tree
import cv2
from sklearn.ensemble import RandomForestClassifier
from sklearn import cross_validation



In [ ]:
x = ML.Matrix_CV_ML("C:/Users/Nuo Xu/Desktop/2018Fall/MLSP/Project/ML/Images/train",230,230)
x.build_ML_matrix()

In [ ]:
clh = RandomForestClassifier(max_depth = 3)
clh.fit(x.global_matrix, x.labels)


In [ ]:
x.labels

In [145]:
cap = cv2.VideoCapture(0)
count = 0

In [146]:
while(True):
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    cv2.rectangle(frame,(170,170),(400,400),(0,255,0),0)
    crop_image = frame[170:400, 170:400]
    height, width, channels = crop_image.shape
    #print(height, width)
    
    blur = cv2.blur(crop_image,(3,3), 0)
    
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    lower_range = np.array([2,0,0])
    upper_range = np.array([16,255,255])
    
    mask = cv2.inRange(hsv,lower_range,upper_range)
    
    skinkernel = np.ones((5,5))
    #Kernel matrices for morphological transformation    
    dilation = cv2.dilate(mask, skinkernel, iterations = 1)
    erosion = cv2.erode(dilation, skinkernel, iterations = 1) 
    
    filtered = cv2.GaussianBlur(erosion, (15,15), 1)
    ret,thresh = cv2.threshold(filtered, 127, 255, 0)
    
    Label_ret, markers = cv2.connectedComponents(thresh)
    num = markers.max()
    
    #print(num)
    ## If the count of pixels less than a threshold, then set pixels to `0`.
    for i in range(1, num+1):
        pts =  np.where(markers == i)
        if len(pts[0]) < 200:
            markers[pts] = 0
    
        
    label_hue = np.uint8(markers.copy())
    
    label_hue = np.where(label_hue != 0, 255, label_hue)
    

    
    
    res = cv2.bitwise_and(crop_image,crop_image,mask = label_hue)
    
    res = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    
    
    # Find contours
    image, contours, hierarchy = cv2.findContours(label_hue, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE )
    
    
    try:
        # Find contour with maximum area
        contour = max(contours, key = lambda x: cv2.contourArea(x))
        
        # Create bounding rectangle around the contour
        x,y,w,h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image,(x,y),(x+w,y+h),(0,0,255),0)
        
        # Find convex hull
        hull = cv2.convexHull(contour)
        
        # Draw contour
        drawing = np.zeros(crop_image.shape,np.uint8)
        cv2.drawContours(drawing,[contour],-1,(0,255,0),0)
        cv2.drawContours(drawing,[hull],-1,(0,0,255),0)
        
        # Find convexity defects
        hull = cv2.convexHull(contour, returnPoints=False)
        defects = cv2.convexityDefects(contour,hull)
        
        # Use cosine rule to find angle of the far point from the start and end point i.e. the convex points (the finger 
        # tips) for all defects
        count_defects = 0
        
        for i in range(defects.shape[0]):
            s,e,f,d = defects[i,0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f][0])

            a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
            b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
            c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
            angle = (math.acos((b**2 + c**2 - a**2)/(2*b*c))*180)/3.14
            
            # if angle > 90 draw a circle at the far point
            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image,far,1,[0,0,255],-1)

            cv2.line(crop_image,start,end,[0,255,0],2)

    except:
        pass
    
    all_image = np.hstack((drawing, crop_image))
    cv2.imshow('Contours', all_image)
    
    #resized_img = thresh.flatten("F").reshape(1,230*230)
    #prediction = np.int64(clh.predict(resized_img))
    #print(prediction)
    
    #name = "C:/Users/Nuo Xu/Desktop/2018Fall/MLSP/Project/ML/Images/hand%d_3.jpg"%count
    #cv2.imwrite(name, thresh)
    
    #cv2.putText(frame, "Gesture Command",(1,250), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255),2,cv2.LINE_AA)
    #hand_image = np.hstack((res, Lable_ret))
    cv2.imshow('hand',res)
    cv2.imshow('hand_binary',label_hue)
    
    cv2.imshow('original_frmae',frame)
    
    count += 1
    
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    

In [147]:
cap.release()
cv2.destroyAllWindows()